In [13]:
import numpy as np

def simulate_disembarking():
    # Initialize
    # Define number of rows and columns
    n_rows = 33
    n_cols = 6

    # Calculate number of passengers
    n_pass = n_rows * n_cols

    # Create seat matrix, using -1 to represent unoccupied seats
    seats = np.zeros((n_rows, n_cols))
    seats[:, :] = -1

    # Create aisle array
    aisle_q = np.zeros(n_rows)
    aisle_q[:] = -1

    # Create initial passenger number queue
    pass_q = [int(i) for i in range(n_pass)]
    pass_q = np.array(pass_q)

    # Account for passenger ages and physical passenger disabilities
    # Define the parameters for the age distribution
    mean_age = 40
    stddev_age = 15

    # Generate passenger ages based on the custom distribution
    passenger_ages = np.random.normal(mean_age, stddev_age, n_pass)
    # Ensure the ages are non-negative
    passenger_ages = np.maximum(passenger_ages, 0)
    # Round the ages to whole numbers
    passenger_ages = np.round(passenger_ages)

    passenger_disabilities = np.random.choice([0, 1], n_pass, p=[0.95, 0.05])  # 5% chance of having a disability

    
    def get_walk_speed(age, disability):
        if age > 70 or age < 8:
            if disability == 1:
                mean_time = 1.2  # 20% increase for age > 70 or < 8 and disability
            else:
                mean_time = 1.2  # 20% increase for age > 70 or < 8 and no disability
        elif disability == 1:
            mean_time = 1.5  # 50% increase for passengers with disabilities
        else:
            mean_time = 1.0  # Default mean_time for other passengers

        stddev_time = 0.2
        walk_speed = np.random.normal(loc=mean_time, scale=stddev_time)
        return walk_speed
    
    
    # Create an array of walking speeds based on age and disability
    walk_speeds = [get_walk_speed(age, disability) for age, disability in zip(passenger_ages, passenger_disabilities)]

    # Create function to move passengers out of the aircraft
    def MoveOutOfAisle(t, aisle_q, pass_q, sum_time):
        if t > sum_time[0]:
            if aisle_q[0] == -1:
                aisle_q[0] = pass_q[0].copy()
                pass_q = np.delete(pass_q, 0)
                sum_time = np.delete(sum_time, 0)
        return aisle_q, pass_q, sum_time

    
    moveto_loc = np.zeros(n_pass)  # Initialize the moveto_loc array
    moveto_time = np.zeros(n_pass)  # Initialize the moveto_time array
    pass_dict = {}  # Define pass_dict
    moveto_loc_dict = {i: j for i, j in zip(pass_q, moveto_loc)}  # Correctly create the initial moveto_loc_dict
    moveto_time_dict = {i: j for i, j in zip(pass_q, moveto_time)}  # Correctly create the initial moveto_time_dict
    
    # Define initial conditions
    time = 0
    time_step = 0.1
    exit_sum = np.sum(pass_q)
    pass_sum = np.sum(seats)
    sum_time = np.zeros(n_pass)  # Initialize the sum_time array

    while pass_sum != exit_sum:
        # Try to move passenger outside the plane if passengers are left
        if pass_q.size != 0:
            aisle_q, pass_q, sum_time = MoveOutOfAisle(time, aisle_q, pass_q, sum_time)

        # Scan the aisle first for non-negative units (passengers)
        for passg in aisle_q:
            if passg != -1:
                # Store the row of the passenger in the aisle
                row = int(np.where(aisle_q == passg)[0][0])
                # See if move has been assigned to the passenger in the aisle
                if moveto_time_dict[passg] != 0:
                    # If move has been assigned, check if it is time to move
                    if time > moveto_time_dict[passg]:
                        # If it is time to move, follow the procedure below
                        # Check if move is backward in the aisle or outside the plane
                        if moveto_loc_dict[passg] == "a":
                            # If move is in the aisle, check if the position behind is empty
                            if aisle_q[row - 1] == -1:
                                # If the position is empty, move the passenger behind and free the position ahead
                                aisle_q[row - 1] = passg
                                aisle_q[row] = -1
                                # Set moves to 0 again
                                moveto_loc_dict[passg] = 0
                                moveto_time_dict[passg] = 0
                        elif moveto_loc_dict[passg] == "o":
                            # If the move is outside the plane,
                            # Find the seat row and column of the passenger
                            passg_row = int(pass_dict[passg][0])
                            passg_col = int(pass_dict[passg][1])
                            # Free the seat position
                            seats[passg_row, passg_col] = -1
                            # Free the aisle
                            aisle_q[row] = -1
                elif moveto_time_dict[passg] == 0:
                    # If the move hasn't been assigned to the passenger
                    # Check the passenger's seat location
                    passg_row = int(pass_dict[passg][0])
                    passg_col = int(pass_dict[passg][1])
                    if passg_row == row:
                        # If the passenger is at the row where his/her seat is
                        # Designate the move type as outside the plane
                        moveto_loc_dict[passg] = "o"
                        # Designate when it is time to move
                        moveto_time_dict[passg] = time + time_dict[passg]
                    elif passg_row != row:
                        # If the passenger is not at the row where his/her seat is
                        # Designate the movement type as aisle
                        moveto_loc_dict[passg] = "a"
                        # Designate the time to move
                        moveto_time_dict[passg] = time + time_dict[passg]

        # Iteration timekeeping
        time += time_step
        pass_sum = np.sum(seats)

    return time



In [15]:
# Number of simulations
num_simulations = 10


boarding_times = []

for _ in range(num_simulations):
    result = simulate_disembarking()
    if result > 0:
        boarding_times.append(result)
    else:
        print("negative number!")

# Calculate the average time
average_time = sum(boarding_times) / len(boarding_times)

# Calculate percentiles
percentile_5 = np.percentile(boarding_times, 5)
percentile_95 = np.percentile(boarding_times, 95)

print("Average disembarking time for {} simulations: {:.2f}".format(len(boarding_times), average_time))
print("5th percentile disembarking time: {:.2f}".format(percentile_5))
print("95th percentile disembarking time: {:.2f}".format(percentile_95))

# Create a histogram of boarding times
plt.hist(boarding_times, bins=20, color='blue', alpha=0.7)
plt.xlabel("Disembarking Time")
plt.ylabel("Frequency")
plt.title("Histogram of Disembarking Times")
plt.grid(True)
plt.show()

KeyError: 0.0

In [ ]:
# Number of simulations
num_simulations = 10


boarding_times = []

for _ in range(num_simulations):
    result = simulate_boarding()
    if result > 0:
        boarding_times.append(result)
    else:
        print("negative number!")

# Calculate the average time
average_time = sum(boarding_times) / len(boarding_times)

# Calculate percentiles
percentile_5 = np.percentile(boarding_times, 5)
percentile_95 = np.percentile(boarding_times, 95)

print("Average disembarking time for {} simulations: {:.2f}".format(len(boarding_times), average_time))
print("5th percentile boarding time: {:.2f}".format(percentile_5))
print("95th percentile boarding time: {:.2f}".format(percentile_95))

# Create a histogram of boarding times
plt.hist(boarding_times, bins=20, color='blue', alpha=0.7)
plt.xlabel("Boarding Time")
plt.ylabel("Frequency")
plt.title("Histogram of Boarding Times")
plt.grid(True)
plt.show()